
<h2><center><font color='black'>   Taking a Deep-Learning Dive with Keras </font></center></h2>

<img src='../imgs/Deep_Dive_Keras_2.jpg' align='middle'/>

<img src='imgs/cnn1.png'/>

<img src='imgs/cnn_family_a.png'/>

<img src='../imgs/cnn_family.png'/>
<img src='../imgs/many_architectures.png'/>
* bubble size is # of parameters
* AlexNet  (62 million parameters) (final layer 6x 6 x 256 x 2096) ~ > 1/2 are coming from FC layer

<img src='imgs/input.png'/>

<img src='../imgs/conv1.png'/>

<img src='../imgs/conv_title.png'/>

<img src='../imgs/3D_Convolution_Animation.gif'/>

**Definition of Convolution:**
Integral that expresses overlap of one function as it is shifted over another

**In terms of CNNs:**
The input is one function, and the kernel is the other. We can implement the integral as 
a summation over a finite # of array elements.  The output is called the feature map (which we can think of as a weighted sum)

<img src='../imgs/conv_a.png'/>
<img src='../imgs/conv_bb.png'/>

<img src='../imgs/conv_layers.png'/>


<img src='imgs/conv_filter.png'/>

<img src='imgs/max_pool.png'/>


### Max-Pooling benefits: 

- Reduces parameters by 75% 
- Helps us to generalize our model: Provides an 'abstracted' form of our features

<img src='../imgs/relu_1.png'/>

<img src='../imgs/relu2.png'/>


<img src='imgs/fc.png'/>

<img src='imgs/all_layers.png'/>

Credits: [Yosinski](http://yosinski.com)




In [3]:
from keras import backend as K
import keras.utils.np_utils

from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D, ZeroPadding1D
from keras.applications.vgg16 import VGG16

from keras. preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import numpy as np
import pandas as pd

from IPython.display import SVG
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import sys

#import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

##  And now,  CNN in Keras! 


In [4]:
import keras.backend as K

#### Macroarchitecture of VGG16
<img src='../imgs/VGG.png'/>

<img src='../imgs/dress_two.png'/>

### Working with CNNs &  [CIFAR dataset ](https://www.cs.toronto.edu/~kriz/cifar.html)
- The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes

<img src ='../imgs/cifar.png'/>

In [2]:
from keras.datasets import cifar10
 
# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
X_train.shape

(50000, 32, 32, 3)

In [4]:
from keras.utils.np_utils import to_categorical
y_test=to_categorical(y_test)
y_train=to_categorical(y_train)

###  Let's work with VGG16 model again

Lets import the model this time



In [16]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights=None, include_top=False)
model_vgg16_conv.summary()

# Tailor our input format (Note- Tensorflow input order!)
input = Input(shape=(32,32,3),name = 'image_input')  # our input shape: (3,32,32)

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(10, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)  # Using the Functional API

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, None, None, 64)1792        input_3[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, None, None, 64)36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, None, None, 64)0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [21]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
my_model.compile(optimizer=sgd, loss='categorical_crossentropy',
                 metrics=['accuracy'])


In [22]:
cb_a=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
# lets look at our results while we are at it
cb_b=keras.callbacks.RemoteMonitor(root='http://localhost:9000')


my_model.fit(X_train, y_train,
          batch_size=128, nb_epoch=10, verbose=1,
          validation_data=(X_test, y_test),callbacks=[cb_a,cb_b])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 235s - loss: 2.3043 - acc: 0.1000 - val_loss: 2.3043 - val_acc: 0.1000
Epoch 2/10
50000/50000 [==============================] - 193s - loss: 2.3043 - acc: 0.0986 - val_loss: 2.3040 - val_acc: 0.1000
Epoch 3/10
50000/50000 [==============================] - 193s - loss: 2.3044 - acc: 0.0997 - val_loss: 2.3033 - val_acc: 0.1000
Epoch 4/10
50000/50000 [==============================] - 193s - loss: 2.3045 - acc: 0.1000 - val_loss: 2.3035 - val_acc: 0.1000
Epoch 5/10
50000/50000 [==============================] - 193s - loss: 2.3043 - acc: 0.0984 - val_loss: 2.3043 - val_acc: 0.1000
Epoch 6/10
50000/50000 [==============================] - 193s - loss: 2.3044 - acc: 0.0987 - val_loss: 2.3044 - val_acc: 0.1000


Our Training has flat-lined!  
Let's try again: 

### Code-Along! 

Let's repeat the MNIST training by leveraging [Transfer Learning](http://cs231n.github.io/transfer-learning/)

1) Reinstantiate your VGG16 model: this time set weights='imagenet'    
2) Freeze all previous layers    
3) We'll want to rescale our image features to be between 0 & 1:   

hint: see Kera's [ImageDataGenerator Function](https://keras.io/preprocessing/image/)



In [4]:
# Let's reload the data
from keras.datasets import cifar10
import cv2
 
# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [7]:
X_train[0].shape

(32, 32, 3)

In [12]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = keras.applications.VGG16(weights='imagenet', include_top=False)
#model_vgg16_conv.summary()

#Create your own input format 
input = Input(shape=(32,32,3),name = 'image_input')  # our input shape: (3,32,32)

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)
    

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(10, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)  # Using the Functional API

In [13]:
# Freeze convolutional layers
for layer in my_model.layers[:2]:
    layer.trainable=False

In [14]:
for layer in my_model.layers:
    print(layer.trainable)

False
False
True
True
True
True


In [18]:
# must use a very small learning rate 
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

my_model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

In [19]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)

test_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=32)

# same callbacks!
cb_a=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
# lets look at our results while we are at it
#cb_b=keras.callbacks.RemoteMonitor(root='http://localhost:9000')

# fine-tune the model
my_model.fit_generator(
    train_generator,
    samples_per_epoch= X_train.shape[0],
    nb_epoch=10,
    validation_data=validation_generator,
    nb_val_samples=  X_test.shape[0],
    callbacks=[cb_a])

Epoch 1/10
50000/50000 [==============================] - 145s - loss: 1.3028 - acc: 0.5445 - val_loss: 1.2546 - val_acc: 0.5595
Epoch 2/10
50000/50000 [==============================] - 140s - loss: 1.2838 - acc: 0.5516 - val_loss: 1.2123 - val_acc: 0.5782
Epoch 3/10
50000/50000 [==============================] - 140s - loss: 1.2628 - acc: 0.5562 - val_loss: 1.2049 - val_acc: 0.5762
Epoch 4/10
50000/50000 [==============================] - 140s - loss: 1.2443 - acc: 0.5649 - val_loss: 1.1921 - val_acc: 0.5790
Epoch 5/10
50000/50000 [==============================] - 140s - loss: 1.2321 - acc: 0.5675 - val_loss: 1.1739 - val_acc: 0.5940
Epoch 6/10
50000/50000 [==============================] - 140s - loss: 1.2204 - acc: 0.5748 - val_loss: 1.1753 - val_acc: 0.5905
Epoch 7/10
50000/50000 [==============================] - 140s - loss: 1.2043 - acc: 0.5775 - val_loss: 1.1757 - val_acc: 0.5818
Epoch 8/10
50000/50000 [==============================] - 140s - loss: 1.1933 - acc: 0.5842 - val